In [12]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

METADATA_FILEPATH = "./../data/articles/articles.parquet"
DATABASE_PATH = "./../qdrant_data"

In [6]:
df = pd.read_parquet(METADATA_FILEPATH)
df.tail()

,newsId,url,publishDate,title,titleCharCount,textCharCount,topics,body
148094,58bc855d-87e3-4e3d-afd0-27c7fae77ac1,http://g1.globo.com/go/goias/noticia/2023/03/2...,2023-03-26 14:40:11,Jovem morre após levar choque ao pular muro de...,86,1678,[go],"Gabriel Feitosa da Silva, de 25 anos, morreu a..."
148095,60b204c2-54c0-447b-a751-eda8f32a2bf3,http://g1.globo.com/sc/santa-catarina/noticia/...,2022-06-21 00:18:26,Corregedoria-Geral da Justiça apura conduta de...,104,4453,[sc],A Corregedoria-Geral da Justiça está investiga...
148096,d7eab297-d17a-4634-ba6e-7699500e78bb,http://g1.globo.com/pe/pernambuco/blog/viver-n...,2022-12-19 17:37:11,\nEstudo relata 1ª evidência de acasalamento d...,105,4551,[pe],Registros de marca de cópula em tigres em Noro...
148097,132fb4d8-1182-4655-91cf-c809e3700642,http://g1.globo.com/ciencia-e-saude/noticia/a-...,2018-03-23 18:16:02,A trágica história por trás da 'múmia de extra...,76,2971,[mundo],"O esqueleto da múmia Ata tem 13 centímetros, m..."
148098,9ff4179c-d58c-4040-920b-eb4c7dbd031b,http://g1.globo.com/ciencia/noticia/2023/03/14...,2023-03-14 11:06:08,Sonda da Nasa flagra dunas de areia 'atípicas'...,55,2059,[ciencia],As dunas de areia circulares vista em uma imag...


In [11]:
def generate_item_sentence(row: pd.Series) -> str:
    return ' '.join([row["title"], row["body"]])

df["sentence"] = df.apply(generate_item_sentence, axis=1)
df.tail()

,newsId,url,publishDate,title,titleCharCount,textCharCount,topics,body,sentence
148094,58bc855d-87e3-4e3d-afd0-27c7fae77ac1,http://g1.globo.com/go/goias/noticia/2023/03/2...,2023-03-26 14:40:11,Jovem morre após levar choque ao pular muro de...,86,1678,[go],"Gabriel Feitosa da Silva, de 25 anos, morreu a...",Jovem morre após levar choque ao pular muro de...
148095,60b204c2-54c0-447b-a751-eda8f32a2bf3,http://g1.globo.com/sc/santa-catarina/noticia/...,2022-06-21 00:18:26,Corregedoria-Geral da Justiça apura conduta de...,104,4453,[sc],A Corregedoria-Geral da Justiça está investiga...,Corregedoria-Geral da Justiça apura conduta de...
148096,d7eab297-d17a-4634-ba6e-7699500e78bb,http://g1.globo.com/pe/pernambuco/blog/viver-n...,2022-12-19 17:37:11,\nEstudo relata 1ª evidência de acasalamento d...,105,4551,[pe],Registros de marca de cópula em tigres em Noro...,\nEstudo relata 1ª evidência de acasalamento d...
148097,132fb4d8-1182-4655-91cf-c809e3700642,http://g1.globo.com/ciencia-e-saude/noticia/a-...,2018-03-23 18:16:02,A trágica história por trás da 'múmia de extra...,76,2971,[mundo],"O esqueleto da múmia Ata tem 13 centímetros, m...",A trágica história por trás da 'múmia de extra...
148098,9ff4179c-d58c-4040-920b-eb4c7dbd031b,http://g1.globo.com/ciencia/noticia/2023/03/14...,2023-03-14 11:06:08,Sonda da Nasa flagra dunas de areia 'atípicas'...,55,2059,[ciencia],As dunas de areia circulares vista em uma imag...,Sonda da Nasa flagra dunas de areia 'atípicas'...


In [126]:

df[[
    "newsId", "title", "body", 
]].head()#.to_clipboard(sep=',', index=None)

,newsId,url,title,titleCharCount,textCharCount,topics,body,sentence,publishTimestamp
12,c840c6a6-626b-419b-996e-73afdee03a29,http://g1.globo.com/sp/sao-carlos-regiao/notic...,Polícia Civil prende três suspeitos envolvidos...,98,3139,[sp],Idosa morre após ser baleada em Vargem Grande ...,Polícia Civil prende três suspeitos envolvidos...,1.682343e+09
172,e33d0e6d-aafd-41f4-bef1-b107b76fc673,http://g1.globo.com/pr/campos-gerais-sul/notic...,Pedreiro é condenado a 14 anos de prisão por m...,100,1912,[pr],Pedreiro enfrenta júri popular por morte de co...,Pedreiro é condenado a 14 anos de prisão por m...,1.682453e+09
306,7150e244-a403-4e63-8831-3e9ca99a4afd,http://g1.globo.com/ap/amapa/noticia/2023/04/2...,Covid-19: veja onde Macapá oferta vacinas em 2...,57,2345,[ap],Vacinação em Macapá\nPMM/Divulgação\nNesta sex...,Covid-19: veja onde Macapá oferta vacinas em 2...,1.682641e+09
336,1eddaedc-c687-4e1e-a70c-18101c6862cb,http://g1.globo.com/politica/noticia/2023/04/2...,Comissão dá aval a projeto para suspender decr...,99,2443,[política],A Comissão de Segurança Pública da Câmara dos ...,Comissão dá aval a projeto para suspender decr...,1.682449e+09
360,51cb0e38-9928-4158-9fa7-097c369e2d83,http://g1.globo.com/sp/sao-paulo/noticia/2023/...,Thiago Brennand chega à sede da PF em São Paul...,98,3603,[são paulo],Brennand chega a São Paulo escoltado pela PF\n...,Thiago Brennand chega à sede da PF em São Paul...,1.682802e+09


In [114]:
sample_sentence = df.query("newsId == '4159a128-f6c0-44a7-b08f-0a17f099d43f'")["sentence"].values[0]
sample_embedding = encoder.encode(sample_sentence)
print (sample_sentence)
print (sample_embedding[:4])


Paraguaios vão às urnas neste domingo (30) para escolher novo presidente Eleição no Paraguai tem disputa acirrada pela Presidência
Os paraguaios vão às urnas neste domingo (30) para escolher o novo presidente do país.
As pesquisas de opinião indicam uma disputa apertada entre o candidato governista Santiago Peña, do Partido Colorado, e o oposicionista Efraín Alegre, do Concertación Nacional, de esquerda. 
A votação é em turno único, e também vai eleger parlamentares e governadores. Dois temas dominaram as discussões na campanha: a corrupção, e a ampliação das relações comerciais com a China.
[-0.2875876   0.0356041   0.31462672  0.06252239]


# Preprocessing

In [25]:
# Sampling
max_days = 7
df = df[df["publishDate"] >= df["publishDate"].max() + pd.DateOffset(-max_days)]

newsId                    object
url                       object
publishDate       datetime64[us]
title                     object
titleCharCount             int64
textCharCount              int64
topics                    object
body                      object
sentence                  object
dtype: object

In [41]:
# Dtype conversion
df["publishTimestamp"] = df["publishDate"].apply(pd.Timestamp.timestamp)
df.drop(columns=["publishDate"], inplace=True)

In [46]:
df["topics"] = df["topics"].apply(lambda x: x.tolist())

/var/folders/pr/2nrw06tn1vb56m13zv7wlrsm0000gn/T/ipykernel_5750/3022238567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["topics"] = df["topics"].apply(lambda x: x.tolist())


In [51]:
df.tail()

,newsId,url,title,titleCharCount,textCharCount,topics,body,sentence,publishTimestamp
147774,955f0e20-6820-4ddc-b0fe-9228c3f00bef,http://g1.globo.com/mg/grande-minas/noticia/20...,"Homem morre com cinco tiros na cabeça, em Pai ...",51,997,[mg],"Um homem, de 36 anos, morreu a tiros na tarde ...","Homem morre com cinco tiros na cabeça, em Pai ...",1.682356e+09
147877,16022899-4880-4505-8b46-db26f016bf00,http://g1.globo.com/politica/blog/andreia-sadi...,CPMI dos atos golpistas: oposição vai mirar em...,80,1520,[política],"General Braga Netto e o ministro da Justiça, F...",CPMI dos atos golpistas: oposição vai mirar em...,1.682343e+09
147992,7f9ad39d-09a1-4764-b3f4-59b9c5f2b6f8,http://g1.globo.com/pe/pernambuco/noticia/2023...,Chuva no Grande Recife deixa ruas alagadas e t...,83,3838,[pe],Ruas ficam alagadas após chuvas no Recife\nRua...,Chuva no Grande Recife deixa ruas alagadas e t...,1.682507e+09
148007,41350c69-0b8a-4f9c-9eac-e6242e6fc0b0,http://g1.globo.com/jornal-nacional/noticia/20...,Venda de carros usados no país passa a ser cin...,107,2517,[jornal-nacional],Venda de carros usados no país passa a ser cin...,Venda de carros usados no país passa a ser cin...,1.682814e+09
148054,6ceee695-f2eb-4541-be38-a7782bd4fdcc,http://g1.globo.com/ba/bahia/noticia/2023/04/2...,Covid-19: veja estratégia de vacinação que inc...,116,14240,[bahia],Vacinação contra Covid-19 segue em Salvador\nB...,Covid-19: veja estratégia de vacinação que inc...,1.682467e+09


# Qdrant Setup

In [18]:
client = QdrantClient(path=DATABASE_PATH)

In [15]:
model_name = "neuralmind/bert-base-portuguese-cased"
collection_name = "news-articles"

encoder = SentenceTransformer(model_name_or_path=model_name, device='cpu')

No sentence-transformers model found with name /Users/joao.guedes/.cache/torch/sentence_transformers/neuralmind_bert-base-portuguese-cased. Creating a new one with MEAN pooling.


Checking collections

In [19]:
client.get_collections()

CollectionsResponse(collections=[])

Creating a collection

In [20]:
from qdrant_client.http.models import Distance, VectorParams

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE,
    ),
)

True

In [21]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='news-articles')])

# Defining Vectors

In [23]:
df.tail()

,newsId,url,publishDate,title,titleCharCount,textCharCount,topics,body,sentence
148094,58bc855d-87e3-4e3d-afd0-27c7fae77ac1,http://g1.globo.com/go/goias/noticia/2023/03/2...,2023-03-26 14:40:11,Jovem morre após levar choque ao pular muro de...,86,1678,[go],"Gabriel Feitosa da Silva, de 25 anos, morreu a...",Jovem morre após levar choque ao pular muro de...
148095,60b204c2-54c0-447b-a751-eda8f32a2bf3,http://g1.globo.com/sc/santa-catarina/noticia/...,2022-06-21 00:18:26,Corregedoria-Geral da Justiça apura conduta de...,104,4453,[sc],A Corregedoria-Geral da Justiça está investiga...,Corregedoria-Geral da Justiça apura conduta de...
148096,d7eab297-d17a-4634-ba6e-7699500e78bb,http://g1.globo.com/pe/pernambuco/blog/viver-n...,2022-12-19 17:37:11,\nEstudo relata 1ª evidência de acasalamento d...,105,4551,[pe],Registros de marca de cópula em tigres em Noro...,\nEstudo relata 1ª evidência de acasalamento d...
148097,132fb4d8-1182-4655-91cf-c809e3700642,http://g1.globo.com/ciencia-e-saude/noticia/a-...,2018-03-23 18:16:02,A trágica história por trás da 'múmia de extra...,76,2971,[mundo],"O esqueleto da múmia Ata tem 13 centímetros, m...",A trágica história por trás da 'múmia de extra...
148098,9ff4179c-d58c-4040-920b-eb4c7dbd031b,http://g1.globo.com/ciencia/noticia/2023/03/14...,2023-03-14 11:06:08,Sonda da Nasa flagra dunas de areia 'atípicas'...,55,2059,[ciencia],As dunas de areia circulares vista em uma imag...,Sonda da Nasa flagra dunas de areia 'atípicas'...


[PointStruct(id='c840c6a6-626b-419b-996e-73afdee03a29', vector=[-0.27530789375305176, -0.10961669683456421, 0.3222742974758148, 0.03172452002763748, 0.24828609824180603, 0.3044049143791199, 0.0015107924118638039, 0.2151155024766922, -0.24580124020576477, -0.23382225632667542, 0.29348456859588623, 0.34965166449546814, -0.029989661648869514, 0.12359942495822906, -0.24013268947601318, -0.03973056375980377, 0.10251688212156296, -0.13129538297653198, 0.011367762461304665, -0.19394224882125854, -0.3198423385620117, 0.14948657155036926, -0.041984427720308304, -0.2876173257827759, -0.5642433166503906, 0.4022168517112732, -0.06133133918046951, -0.07956783473491669, -0.3409377336502075, -0.5598554611206055, -0.07079695165157318, 0.3145569860935211, -0.1660841554403305, 0.3850100636482239, 0.1314176321029663, 0.07979965209960938, 0.20599853992462158, 0.14139355719089508, 0.08481939136981964, 0.16968804597854614, 0.16847702860832214, 0.0034861043095588684, -0.02792542055249214, 0.03136676549911499

In [80]:
from qdrant_client.http.models import PointStruct

metadata_columns = df.drop(["newsId", "sentence"], axis=1).columns

def create_vector_point(row:pd.Series) -> PointStruct:
    return PointStruct(
        id=row["newsId"],
        vector=encoder.encode(row["sentence"]).tolist(),
        payload={
            field: row[field]
            for field in metadata_columns
            if (str(row[field]) not in ['None', 'nan'])
        }
    )

points = df.apply(create_vector_point, axis=1).tolist()


0it [00:00, ?it/s]

3558it [3:08:42,  3.18s/it] 


# Uploading Vectors

In [81]:
CHUNK_SIZE = 500
n_chunks = np.ceil(len(points)/CHUNK_SIZE)

for i, points_chunk in enumerate(np.array_split(points, n_chunks)):
    print (f"Processing chunk {i+1}/{int(n_chunks)}")
    operation_info = client.upsert(
        collection_name=collection_name,
        wait=True,
        points=points_chunk.tolist()
    )

Processing chunk 1/8
Processing chunk 2/8
Processing chunk 3/8
Processing chunk 4/8
Processing chunk 5/8
Processing chunk 6/8
Processing chunk 7/8
Processing chunk 8/8


In [82]:
client.get_collection(collection_name).vectors_count

3558

# Querying Database

In [84]:
# https://qdrant.tech/documentation/concepts/filtering/
from qdrant_client.models import Filter
from qdrant_client.http import models

client.scroll(
    collection_name=collection_name,
    limit=5,
    with_payload=["newsId", "title", "topics"],
    with_vectors=False,
    scroll_filter=Filter(
        must=[
            models.FieldCondition(key="topics", match=models.MatchAny(any=["política", "mundo"])),
        ]
    )
)

([Record(id='004b22c3-87f5-4270-b6bb-4ddfc8490f2a', payload={'title': 'Lira manda recado após veto de indicado para o Tribunal Regional Federal da 1ª Região', 'titleCharCount': 85, 'topics': ['política']}, vector=None),
  Record(id='007fa598-ce88-42a5-b63b-c6810614e663', payload={'title': 'Moraes vota para tornar réus 200 denunciados de incitar e executar atos golpistas de 8 de janeiro', 'titleCharCount': 97, 'topics': ['política']}, vector=None),
  Record(id='00b89b4c-d912-4394-84de-97a94abe8ca9', payload={'title': 'Terremoto de magnitude 5,7 atinge costa de Valparaíso, no Chile', 'titleCharCount': 63, 'topics': ['mundo']}, vector=None),
  Record(id='0114fb71-893f-4a84-839b-5a9b3ddd2b38', payload={'title': 'Projetos do governo passam a travar pauta da Câmara na próxima semana; veja quais', 'titleCharCount': 81, 'topics': ['política']}, vector=None),
  Record(id='0152157d-a7cc-48be-9bda-a5c2b0ae9207', payload={'title': 'Mortos em seita de jejum no Quênia chegam a 73', 'titleCharCount':

In [94]:
# https://qdrant.tech/documentation/concepts/filtering/
from qdrant_client.models import Filter
from qdrant_client.http import models

query_text = "Donald Trump"
query_vector = encoder.encode(query_text).tolist()

client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=10,
    with_payload=["newsId", "title", "topics"],
    with_vectors=False,
    score_threshold=0,
    query_filter=None
)

[ScoredPoint(id='4159a128-f6c0-44a7-b08f-0a17f099d43f', version=0, score=0.3904202143986567, payload={'title': 'Paraguaios vão às urnas neste domingo (30) para escolher novo presidente', 'titleCharCount': 72, 'topics': ['jornal-nacional']}, vector=None),
 ScoredPoint(id='e4db00c9-7bcb-4c88-a502-4c6c840dde30', version=0, score=0.38603588889463514, payload={'title': 'Eleitores dizem que Biden e Trump não deveriam concorrer em 2024, mostra pesquisa Reuters/Ipsos', 'titleCharCount': 95, 'topics': ['mundo']}, vector=None),
 ScoredPoint(id='8efe078b-9f42-446e-a970-9abfe408bb84', version=0, score=0.3775547801532631, payload={'title': 'Escritora acusa Trump de ter abusado sexualmente dela nos anos 1990', 'titleCharCount': 67, 'topics': ['jornal-nacional']}, vector=None),
 ScoredPoint(id='d2f89819-e36d-4e23-a13c-47c582fe1363', version=0, score=0.37731883793134285, payload={'title': 'Mike Pence, ex-vice de Donald Trump, presta depoimento na Justiça que pode complicar o ex-presidente', 'titleChar

In [89]:

client.recommend(
    collection_name=collection_name,
    positive=['8bc22460-532c-449b-ad71-28dd86790ca2'],
    limit = 10,
    with_payload=["newsId", "title", "topics"],
    with_vectors=False,
    score_threshold=0,
    query_filter=None
)

[ScoredPoint(id='6087ba13-40d7-4f44-ae77-2abb26e6193e', version=0, score=0.9814363121986389, payload={'title': 'Dólar fecha em alta, em linha com o exterior e de olho em Campos Neto no Senado', 'titleCharCount': 79, 'topics': ['economia']}, vector=None),
 ScoredPoint(id='77a71db8-cffb-4260-9ca8-f8221951c301', version=0, score=0.9806766510009766, payload={'title': 'Dólar cai e volta a fechar abaixo de R$ 5, após PIB fraco nos EUA e de olho em falas de Haddad e Campos Neto ', 'titleCharCount': 109, 'topics': ['economia']}, vector=None),
 ScoredPoint(id='2a3e52aa-b1d0-4abe-b0d4-de5898eef1a9', version=0, score=0.9711627960205078, payload={'title': 'Ibovespa recua e fecha aos 103 mil pontos, puxado por Vale', 'titleCharCount': 58, 'topics': ['economia']}, vector=None),
 ScoredPoint(id='138d0e41-6d9f-4b6f-bc7b-e568adfaa213', version=0, score=0.967682957649231, payload={'title': 'Dólar sobe nesta sexta-feira, mas fecha o mês abaixo de R$ 5', 'titleCharCount': 60, 'topics': ['economia']}, vect

In [120]:
df.query('newsId == "9ed25e49-14f9-44cd-b76e-a104b4dfabca"')["title"].tolist()[0]

'Saiba por que Joe Biden lançou sua candidatura à reeleição nesta terça-feira'

In [95]:
seed_ids = ["9ed25e49-14f9-44cd-b76e-a104b4dfabca"]

seed_records = client.scroll(
    collection_name=collection_name,
    limit=5,
    with_payload=["newsId", "title", "topics"],
    with_vectors=False,
    scroll_filter=Filter(
        must=[
            models.HasIdCondition(has_id=seed_ids),
        ]
    )
)[0]

seed_records

[Record(id='9ed25e49-14f9-44cd-b76e-a104b4dfabca', payload={'title': 'Saiba por que Joe Biden lançou sua candidatura à reeleição nesta terça-feira', 'titleCharCount': 76, 'topics': ['mundo']}, vector=None)]

In [96]:
match_key = "topics"

client.recommend(
    collection_name=collection_name,
    positive=[record.id for record in seed_records],
    limit = 10,
    with_payload=["newsId", "title", "topics"],
    with_vectors=False,
    score_threshold=0,
    query_filter=Filter(
        should=[
            models.FieldCondition(key=match_key, match=models.MatchAny(any=record.payload[match_key]))
            for record in seed_records
        ]
    )
)

[ScoredPoint(id='38a1cd6f-aa07-4264-8ca1-4cddf05e7afe', version=0, score=0.9734960794448853, payload={'title': 'Biden anuncia que vai concorrer à reeleição', 'titleCharCount': 43, 'topics': ['mundo']}, vector=None),
 ScoredPoint(id='4619ee33-07b6-4121-8724-cdc4905bd962', version=0, score=0.9709303975105286, payload={'title': 'EUA: os 4 motivos que levaram Biden a se lançar à reeleição', 'titleCharCount': 59, 'topics': ['mundo']}, vector=None),
 ScoredPoint(id='e4db00c9-7bcb-4c88-a502-4c6c840dde30', version=0, score=0.960002064704895, payload={'title': 'Eleitores dizem que Biden e Trump não deveriam concorrer em 2024, mostra pesquisa Reuters/Ipsos', 'titleCharCount': 95, 'topics': ['mundo']}, vector=None),
 ScoredPoint(id='7f45e444-b220-47e4-933a-ac6dc3074f56', version=0, score=0.9580975770950317, payload={'title': 'A gafe de assessora de Biden que gerou dúvidas sobre eventual segundo governo após eleição', 'titleCharCount': 90, 'topics': ['mundo']}, vector=None),
 ScoredPoint(id='d2f89